In [ ]:
# !Convert*overall_point.py*w*sehx*

In [ ]:
# !Switch*
from handler import FigureHandler
from dataset import ot_2d_random, ot_2d_ellipse, ot_2d_Caffarelli
from stats import Statistics
# !SwitchCase*
# import font
# from handler import FigureHandler
# from dataset import ot_2d_random, ot_2d_ellipse, ot_2d_Caffarelli
# from stats import Statistics
# !SwitchEnd*

In [ ]:
from solver_mosek import solve_mosek_primal_simplex, solve_mosek_dual_simplex, solve_mosek_interior_point
from solver_gurobi import solve_gurobi_primal_simplex, solve_gurobi_dual_simplex, solve_gurobi_barrier
from first_ADMM_primal import solve_ADMM_primal
from first_grad_primal import solve_grad_primal, solve_combine
from discrete_transportation_simplex import solve_transportation_simplex
from multiscale_point_mosek import solve_multiscale_point_mosek_primal_simplex

In [ ]:
# !Switch*
fh = FigureHandler(redir=True)
# !SwitchCase*
# fh = FigureHandler(sav=True, disp=False, ext=".pgf", redir=True)
# !SwitchEnd*

In [ ]:
def suite(stat, size):
    stat.set_objx(solve_mosek_interior_point, prog=fh.write)
    stat.test(solve_mosek_primal_simplex, prog=fh.write)
    stat.output_last()
    stat.test(solve_mosek_dual_simplex, prog=fh.write)
    stat.output_last()
    stat.test(solve_mosek_interior_point, prog=fh.write)
    stat.output_last()
    stat.test(solve_gurobi_primal_simplex, prog=fh.write)
    stat.output_last()
    stat.test(solve_gurobi_dual_simplex, prog=fh.write)
    stat.output_last()
    stat.test(solve_gurobi_barrier, prog=fh.write)
    stat.output_last()
    stat.test(
        solve_combine,
        cfgs=[
            {
                "func": solve_ADMM_primal,
                "args": (),
                "kwargs": dict(
                    its=[5000],
                    rhos=[3.],
                    alphas=[1.618],
                    epss=[1e-3],
                )
            },
            {
                "func": solve_grad_primal,
                "args": (),
                "kwargs": dict(
                    its=[max(size, 500), max(size, 500), max(size, 500)],
                    pi0s=[10., 10., 10.],
                    pi1s=[1000., 10000., 100000.],
                    pi2s=[1000., 10000., 100000.],
                    lrs=[1e-6, 1e-7, 1e-8],
                )
            },
        ],
        prog=fh.write,
    )
    stat.output_last()
    stat.test(
        solve_transportation_simplex,
        eps=1e-12,
        it=15000,
        prog=fh.write,
    )
    stat.output_last()
    stat.test(
        solve_multiscale_point_mosek_primal_simplex,
        step=4, shrinks=[50, 25, 10, 0],
        caps=[1., 0.3, 0.1, 0.1],
        errs=[0.001, 0.001, 0.001, 0.00],
        prog=fh.write,
    )
    stat.output_last()

In [ ]:
rep = 1

In [ ]:
size_list = [100, 250, 500, 750, 1000, 1250, 1500, 1750, 2000]

In [ ]:
for size in size_list:
    stat = Statistics(
        probs=[
            ot_2d_random(size, size, i) for i in range(1, rep+1)
        ],
        prob="random {0}*{0}".format(size),
        log=fh.write,
    )
    suite(stat, size)

In [ ]:
for size in size_list:
    stat = Statistics(
        probs=[
            ot_2d_Caffarelli(size, size, i) for i in range(1, rep+1)
        ],
        prob="Caffarelli {0}*{0}".format(size),
        log=fh.write,
    )
    suite(stat, size)

In [ ]:
for size in size_list:
    stat = Statistics(
        probs=[
            ot_2d_ellipse(size, size, i) for i in range(1, rep+1)
        ],
        prob="ellipse {0}*{0}".format(size),
        log=fh.write,
    )
    suite(stat, size)

In [ ]:
# !ConvertEnd*